In [1]:
#pip install -U sentence-transformers

In [2]:
#pip install -e .

In [1]:
from sentence_transformers import SentenceTransformer, models, util, InputExample, losses, SentencesDataset
from torch.utils.data import DataLoader
import torch, os

In [2]:
torch.cuda.empty_cache()

In [5]:
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3,4,5,6,7"

In [6]:
model = SentenceTransformer('distilbert-base-nli-mean-tokens')

#Our sentences we like to encode
sentences = ['This framework generates embeddings for each input sentence',
    'Sentences are passed as a list of string.', 
    'The quick brown fox jumps over the lazy dog.']

#Sentences are encoded by calling model.encode()
sentence_embeddings = model.encode(sentences)

#Print the embeddings
for sentence, embedding in zip(sentences, sentence_embeddings):
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print("")

Sentence: This framework generates embeddings for each input sentence
Embedding: [-4.99963462e-01  2.00929478e-01  4.79733139e-01 -5.45873828e-02
 -3.20265472e-01 -1.18350282e-01  1.19828153e+00  6.53360263e-02
  2.56169170e-01 -6.80206954e-01 -2.65210330e-01  9.71703529e-02
 -2.19010353e-01 -4.23998207e-01 -1.44663639e-02 -2.27258131e-01
 -3.66329372e-01  4.29629609e-02 -2.45088279e-01 -1.15445757e+00
  1.01486564e+00 -4.86819208e-01 -3.97847056e-01  1.33469379e+00
  7.27640569e-01 -2.61947662e-01  9.74692285e-01  3.20754200e-01
  9.10773352e-02 -1.66287750e-01 -1.71081051e-01 -2.45369360e-01
 -2.78276622e-01  5.85471034e-01  2.47854009e-01 -5.16082823e-01
  8.40411127e-01 -1.11888099e+00 -1.02041543e+00 -2.46359974e-01
  3.75287503e-01 -4.89004403e-01  5.48438013e-01 -1.94785863e-01
 -3.49307120e-01  1.85093060e-02 -3.58533785e-02  3.46166164e-01
 -1.75415814e+00 -3.57942224e-01 -1.20965230e+00  2.49583349e-01
  7.49337852e-01 -3.55312288e-01 -5.63870490e-01 -8.27812999e-02
  2.31488

In [7]:
from torch import nn

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model= nn.DataParallel(model)
model.to(device)

DataParallel(
  (module): SentenceTransformer(
    (0): Transformer(
      (auto_model): DistilBertModel(
        (embeddings): Embeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (transformer): Transformer(
          (layer): ModuleList(
            (0): TransformerBlock(
              (attention): MultiHeadSelfAttention(
                (dropout): Dropout(p=0.1, inplace=False)
                (q_lin): Linear(in_features=768, out_features=768, bias=True)
                (k_lin): Linear(in_features=768, out_features=768, bias=True)
                (v_lin): Linear(in_features=768, out_features=768, bias=True)
                (out_lin): Linear(in_features=768, out_features=768, bias=True)
              )
              (sa_layer_norm): LayerNorm((768,), eps=1e-12, ele

In [9]:
#Sentences are encoded by calling model.encode()
emb1 = model.module.encode("witten")
emb2 = model.module.encode("seiberg")

cos_sim = util.pytorch_cos_sim(emb1, emb2)
print("Cosine-Similarity:", cos_sim)

Cosine-Similarity: tensor([[0.8202]])


In [10]:
#Sentences are encoded by calling model.encode()
emb1 = model.module.encode("witten")
emb2 = model.module.encode("maldacena")

cos_sim = util.pytorch_cos_sim(emb1, emb2)
print("Cosine-Similarity:", cos_sim)

Cosine-Similarity: tensor([[0.8127]])


In [11]:
#Sentences are encoded by calling model.encode()
emb1 = model.module.encode("witten")
emb2 = model.module.encode("shyani")

cos_sim = util.pytorch_cos_sim(emb1, emb2)
print("Cosine-Similarity:", cos_sim)

Cosine-Similarity: tensor([[0.7367]])


In [13]:
import pandas as pd

In [14]:
datapd=pd.read_json("/home/ubuntu/project/arxiv-metadata-oai-snapshot.json",  lines=True)

In [15]:
datapd_hep_th = datapd.loc[datapd['categories'].str.contains('hep-th')]
hep_abstracts = datapd_hep_th[ list(datapd_hep_th.iloc[:,0:1]) + ['abstract'] + ['title']]

In [16]:
texttxt=hep_abstracts.abstract.values

In [17]:
text_txt=[x.replace('\n',' ') for x in texttxt]
text_sep = [x.strip().split(".") for x in text_txt]
text_sep_fin = [x[:-1] for x in text_sep]

In [18]:
text_sep_fin[1]

['We give a prescription for how to compute the Callias index, using as regulator an exponential function',
 ' We find agreement with old results in all odd dimensions',
 ' We show that the problem of computing the dimension of the moduli space of self-dual strings can be formulated as an index problem in even-dimensional (loop-)space',
 ' We think that the regulator used in this Letter can be applied to this index problem']

In [19]:
import random
import numpy as np

In [20]:
random.choice(random.choice(text_sep_fin))

' It is shown that thermodynamic quantities largely depend upon the approach to calculate them, boundary conditions for the scalar fields and regularization method'

In [21]:
len(text_sep_fin)

140500

In [22]:
text_sep_fin[2]

['We study space-time symmetries in scalar quantum field theory (including interacting theories) on static space-times',
 ' We first consider Euclidean quantum field theory on a static Riemannian manifold, and show that the isometry group is generated by one-parameter subgroups which have either self-adjoint or unitary quantizations',
 ' We analytically continue the self-adjoint semigroups to one-parameter unitary groups, and thus construct a unitary representation of the isometry group of the associated Lorentzian manifold',
 ' The method is illustrated for the example of hyperbolic space, whose Lorentzian continuation is Anti-de Sitter space']

In [25]:
print(text_sep_fin[4])
print(text_sep_fin[4][0:4])
print(text_sep_fin[4][4:9])

['Possible (algebraic) commutation relations in the Lagrangian quantum theory of free (scalar, spinor and vector) fields are considered from mathematical view-point', ' As sources of these relations are employed the Heisenberg equations/relations for the dynamical variables and a specific condition for uniqueness of the operators of the dynamical variables (with respect to some class of Lagrangians)', ' The paracommutation relations or some their generalizations are pointed as the most general ones that entail the validity of all Heisenberg equations', ' The simultaneous fulfillment of the Heisenberg equations and the uniqueness requirement turn to be impossible', ' This problem is solved via a redefinition of the dynamical variables, similar to the normal ordering procedure and containing it as a special case', ' That implies corresponding changes in the admissible commutation relations', ' The introduction of the concept of the vacuum makes narrow the class of the possible commutatio

In [30]:
random.choice((0,1))

1

## Below we create the dataset of sentences. Nearest neighbours sentences get a similarity score of 1. Next to nearest neighbours get 0.8. A random pair gets 0.##

In [32]:
count=0
train_examples = [InputExample(texts=['My first sentence', 'My second sentence'], label=1.0)]
for i in range(len(text_sep_fin)):
    j=0
    k=0
    for k in range(2):
        while j+k+1 < len(text_sep_fin[i]):
            sent1=text_sep_fin[i][j]
            sent2=text_sep_fin[i][j+1+k]
            j=j+2
            #print("there")
            labeler=(1/2.0)**(k/3.12)
            train_examples.append(InputExample(texts=[sent1, sent2], label=labeler))
        j=0
    k=0
    z=0  
    while j<len(text_sep_fin[i]):
        if random.choice((0,0,0,0,0,1,1,1)):
            while z<2:
                x=random.choice(text_sep_fin)
                #print(x)
                if len(x)>0 and len(text_sep_fin[i][j])>0:
                    sent1=text_sep_fin[i][j]
                    sent2= random.choice(x)
                    labeler=0.0
                    #print("here")
                    count=count+1
                    train_examples.append(InputExample(texts=[sent1, sent2], label=labeler))
                z=z+1
        j=j+1
        z=0
    j=0
    if i%10000==0:
        print(i)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000


# Of the 1.1 million sentence pairs, 542K consist of a pair of random sentences ##

In [33]:
print(len(train_examples))
print(count)

1127885
542165


In [34]:
for i in range(10):
    rand=random.choice(range(1060000))
    print(train_examples[rand],"\n")

<InputExample> label: 0.0, texts: We review a construction of holographic geometries dual to N=4 SYM theory on a Friedmann-Robertson-Walker background and in the presence or absence of a gluon condensate and instanton density;  In this sense, this theory can be seen as a classical implementation of quantum effects in a curved background space-time 

<InputExample> label: 0.0, texts:  We find that the equilibration time associated with the lowest quasinormal mode decreases in a dense medium;  The partition function is computed, and is shown to be a three-manifold invariant generalizing the Casson invariant 

<InputExample> label: 0.8007851798191048, texts:  On the other hand, it is formally equal to the limit $\xi \to 0$ of the renormalizable R$_{\xi}$-gauge;  An apparent conflict between the unitary gauge and the limit $\xi \to 0$ of the R$_{\xi}$-gauge is resolved, and it is demonstrated that results for physical quantities can be obtained in the unitary gauge 

<InputExample> label: 

In [35]:
import torch

In [36]:
print(torch.cuda.memory_summary(device=None, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |  260336 KB |  261624 KB |  289484 KB |   29148 KB |
|       from large pool |  260096 KB |  260096 KB |  260096 KB |       0 KB |
|       from small pool |     240 KB |    1528 KB |   29388 KB |   29148 KB |
|---------------------------------------------------------------------------|
| Active memory         |  260336 KB |  261624 KB |  289484 KB |   29148 KB |
|       from large pool |  260096 KB |  260096 KB |  260096 KB |

In [37]:
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=512)
train_loss = losses.CosineSimilarityLoss(model)

In [38]:
print(train_examples[18351])

<InputExample> label: 0.8007851798191048, texts: A quantum theory representations of real (R) and complex (C) numbers is given that is based on states of single, finite strings of qukits for any base k > 1;  Cauchy sequences of qukit string states are defined from the arithmetic properties


In [40]:
model.module.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100)

In [81]:
model.module.save("/home/ubuntu/sbert_model_mar12/")

In [41]:
#Sentences are encoded by calling model.encode()
emb1 = model.module.encode("witten")
emb2 = model.module.encode("seiberg")

cos_sim = util.pytorch_cos_sim(emb1, emb2)
print("Cosine-Similarity:", cos_sim)

Cosine-Similarity: tensor([[0.9495]])


In [42]:
#Sentences are encoded by calling model.encode()
emb1 = model.module.encode("witten seiberg superysmmetry")
emb2 = model.module.encode("dog cat giraffe")

cos_sim = util.pytorch_cos_sim(emb1, emb2)
print("Cosine-Similarity:", cos_sim)

Cosine-Similarity: tensor([[0.5464]])


In [83]:
#Sentences are encoded by calling model.encode()
emb1 = model.module.encode("witten")
emb2 = model.module.encode("shyani")

cos_sim = util.pytorch_cos_sim(emb1, emb2)
print("Cosine-Similarity:", cos_sim)

Cosine-Similarity: tensor([[0.9369]])


In [43]:
#Sentences are encoded by calling model.encode()
emb1 = model.module.encode("witten")
emb2 = model.module.encode("maldacena")

cos_sim = util.pytorch_cos_sim(emb1, emb2)
print("Cosine-Similarity:", cos_sim)

Cosine-Similarity: tensor([[0.9328]])


In [48]:
model.module.max_seq_length

128

In [49]:
sentences=text_sep_fin[4]
print(sentences)
sentences=' '.join(text_sep_fin[4])
print(sentences)

['Possible (algebraic) commutation relations in the Lagrangian quantum theory of free (scalar, spinor and vector) fields are considered from mathematical view-point', ' As sources of these relations are employed the Heisenberg equations/relations for the dynamical variables and a specific condition for uniqueness of the operators of the dynamical variables (with respect to some class of Lagrangians)', ' The paracommutation relations or some their generalizations are pointed as the most general ones that entail the validity of all Heisenberg equations', ' The simultaneous fulfillment of the Heisenberg equations and the uniqueness requirement turn to be impossible', ' This problem is solved via a redefinition of the dynamical variables, similar to the normal ordering procedure and containing it as a special case', ' That implies corresponding changes in the admissible commutation relations', ' The introduction of the concept of the vacuum makes narrow the class of the possible commutatio

In [50]:
embeddings = model.module.encode(sentences)
embeddings1 = model.module.encode(sentences)
print("Embedding:", embeddings)
print("\n")
print(embeddings+embeddings1)

Embedding: [ 9.24564004e-02  7.12060630e-01 -5.88010967e-01  1.66881055e-01
 -2.61479318e-01 -3.54792714e-01  9.06798482e-01 -3.61955196e-01
  1.50380522e-01  2.59193033e-01 -1.55331984e-01  4.65895742e-01
 -2.65276313e-01 -8.87878358e-01 -8.93679917e-01  2.02902421e-01
 -3.39269668e-01 -8.27996075e-01 -7.17628121e-01 -5.11019886e-01
  1.04681812e-01  7.83849716e-01  1.12894304e-01  2.40331620e-01
 -3.35458249e-01 -9.80436131e-02  2.43844837e-01 -5.97934484e-01
  1.03269100e-01 -9.49371517e-01 -6.39867544e-01 -2.99330801e-04
 -7.55637288e-02  4.73927319e-01  1.87448204e-01  2.00225621e-01
  3.63994896e-01 -1.46306157e-02 -3.03497404e-01 -7.21471012e-01
  3.98245454e-03 -8.02831054e-02  5.74265718e-01  7.26714209e-02
  1.63400009e-01 -8.32860529e-01 -5.61169870e-02  1.91074729e-01
 -8.20930004e-01 -4.76212323e-01 -3.42124164e-01 -6.20498598e-01
  3.34751099e-01 -1.08799577e-01 -2.97612939e-02  4.62280214e-01
 -4.83663499e-01 -7.04208672e-01 -1.09082937e-01  4.82482202e-02
  1.77778006e+

In [51]:
np.average(model.module.encode([sentences,sentences]),axis=0)

array([ 9.24564004e-02,  7.12060630e-01, -5.88010967e-01,  1.66881055e-01,
       -2.61479318e-01, -3.54792714e-01,  9.06798482e-01, -3.61955196e-01,
        1.50380522e-01,  2.59193033e-01, -1.55331984e-01,  4.65895742e-01,
       -2.65276313e-01, -8.87878358e-01, -8.93679917e-01,  2.02902421e-01,
       -3.39269668e-01, -8.27996075e-01, -7.17628121e-01, -5.11019886e-01,
        1.04681812e-01,  7.83849716e-01,  1.12894304e-01,  2.40331620e-01,
       -3.35458249e-01, -9.80436131e-02,  2.43844837e-01, -5.97934484e-01,
        1.03269100e-01, -9.49371517e-01, -6.39867544e-01, -2.99330801e-04,
       -7.55637288e-02,  4.73927319e-01,  1.87448204e-01,  2.00225621e-01,
        3.63994896e-01, -1.46306157e-02, -3.03497404e-01, -7.21471012e-01,
        3.98245454e-03, -8.02831054e-02,  5.74265718e-01,  7.26714209e-02,
        1.63400009e-01, -8.32860529e-01, -5.61169870e-02,  1.91074729e-01,
       -8.20930004e-01, -4.76212323e-01, -3.42124164e-01, -6.20498598e-01,
        3.34751099e-01, -

In [53]:
import time
setnencenp=np.zeros((140500,2))

print(time.time())
for i in range(1000):
    full=len(text_sep_fin[i])
    if full%2 !=0:
        half=np.int(np.floor(full/2))
        sent1=text_sep_fin[i][0:half]
        sent1=' '.join(sent1)      
        sentencenp[i][0]=sent1
        sentencenp[i][1]=sent2
    else:
        half=np.int(np.floor(full/2))
        sent4=text_sep_fin[i][0:half]
        sent4=' '.join(sent4)
        sent5=text_sep_fin[i][half:full]
        sent5=' '.join(sent5)       
        sentemb=np.average(model.module.encode([sent4,sent5]),axis=0)
    if i%10000==0:
        print(i)
print(time.time())

In [54]:
iden=hep_abstracts['id'].values
iden[0]
import time
len(text_sep_fin[0])

2

In [55]:
print(train_examples[3])

<InputExample> label: 0.0, texts:  This paper explores different methods to evaluate these integrals and then uses them to calculate the kinematic factors of the one-loop and two-loop massless four-point amplitudes involving two and four Ramond states;  In particular, we observe a Josephson-type tunneling transition of a wave packet, the effect of its partial trapping and a non-monotonic dependence of tunneling dynamics on the shape of a three-well potential


In [56]:
text_sep_fin[1]

['We give a prescription for how to compute the Callias index, using as regulator an exponential function',
 ' We find agreement with old results in all odd dimensions',
 ' We show that the problem of computing the dimension of the moduli space of self-dual strings can be formulated as an index problem in even-dimensional (loop-)space',
 ' We think that the regulator used in this Letter can be applied to this index problem']

In [57]:
model.module.encode(text_sep_fin[1][1])

array([-4.14200544e-01, -8.00772965e-01,  6.40848875e-01,  1.24949110e+00,
       -3.48137230e-01,  1.67665482e-02, -2.03405414e-02, -2.40527749e-01,
        2.12748900e-01,  4.09965634e-01,  7.55983293e-01,  2.73611933e-01,
        6.75211191e-01, -5.66387117e-01, -5.76862991e-02,  1.39755154e+00,
        1.56209514e-01,  3.17232646e-02,  1.47478536e-01,  1.30724356e-01,
        3.67189527e-01, -5.19806504e-01,  7.98422098e-03,  3.71132255e-01,
       -1.35630950e-01, -4.57002074e-01,  9.44362462e-01, -1.77175403e-01,
       -5.95361829e-01, -1.23522401e-01,  2.63627499e-01, -4.24323708e-01,
        4.60033894e-01,  4.26642805e-01,  3.32565516e-01, -2.33951986e-01,
        3.45568657e-01, -6.09106600e-01,  1.67388454e-01, -8.65928471e-01,
        2.14368045e-01, -5.28290510e-01,  2.27587506e-01, -6.22591674e-01,
       -6.91560924e-01,  3.23897570e-01, -2.66353905e-01,  5.69240212e-01,
       -7.62759924e-01, -4.45090532e-02, -1.34964716e+00,  7.61886895e-01,
        8.27997863e-01,  

In [58]:
model.module.encode(text_sep_fin[0][1])

array([ 1.03648372e-01, -3.15641791e-01, -4.93921608e-01,  1.19470203e+00,
       -6.34805918e-01,  1.94808275e-01,  2.14459747e-01, -6.47991836e-01,
        2.36545473e-01,  1.07012117e+00,  2.23084733e-01,  4.08497393e-01,
        1.15630463e-01, -1.29884374e+00, -5.95215619e-01,  1.30758977e+00,
        6.35598451e-02, -5.04338324e-01,  3.74747157e-01,  5.24180472e-01,
        9.78960097e-01,  1.10706456e-01, -2.58811444e-01,  6.37044966e-01,
        3.42142701e-01, -6.23951435e-01,  1.45808697e-01, -5.72872102e-01,
       -8.58974040e-01,  5.25240926e-03, -1.92970842e-01, -1.81403697e-01,
        5.45322239e-01,  2.20187783e-01, -9.48021770e-01,  9.37144011e-02,
        3.98179561e-01, -3.77459228e-01,  3.72652382e-01, -3.84355009e-01,
       -4.79676455e-01, -2.98443019e-01,  7.81488001e-01, -1.06563914e+00,
        1.12799399e-01, -9.46979076e-02, -1.06598668e-01,  4.84666824e-01,
       -4.50474739e-01,  5.12182653e-01, -1.79655528e+00,  4.03049350e-01,
        4.58566666e-01,  

In [64]:
x0=model.module.encode(text_sep_fin[22353])
x1=model.module.encode(text_sep_fin[63452])
x0=np.average(x0,axis=0)
x1=np.average(x1,axis=0)
(np.dot(x0,x1)**2)/(np.dot(x0,x0)*np.dot(x1,x1))

0.006311861732261465

In [65]:
text_sep_fin[0]

['The pure spinor formulation of the ten-dimensional superstring leads to manifestly supersymmetric loop amplitudes, expressed as integrals in pure spinor superspace',
 ' This paper explores different methods to evaluate these integrals and then uses them to calculate the kinematic factors of the one-loop and two-loop massless four-point amplitudes involving two and four Ramond states']

In [66]:
model.module.encode(text_sep_fin[3]).shape

(4, 768)

In [67]:
model.module.encode(text_sep_fin[13]).shape

(6, 768)

In [ ]:
import os
output_dir = '/home/ubuntu/sbert_model_mar12/'
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Printing the document vectors to a file. The first word of every line is the arxiv ID and the rest if the 768 dimensional document vector #

In [74]:
f = open("/home/ubuntu/project/SBERT/abstracts_sbert_mar12_smallersentences_moredata", "w")
print(time.time())
for i in range(140500):
    if len(text_sep_fin[i])>0:
        x=model.module.encode(text_sep_fin[i])
        x=np.average(x,axis=0)
        f.write(f"{iden[i]} ")
        f.write((' '.join(['%0.10f']*x.size)+'\n') % tuple(x))
        if i%1000==0:
            print(i)
f.close()

1615587103.3048651
0
1615587103.3312368
1000
1615587130.5003917
2000
1615587158.752389
3000
1615587185.4885793
4000
1615587213.326288
5000
1615587241.004158
6000
1615587268.6159787
7000
1615587296.7592592
8000
1615587324.8546422
9000
1615587353.6123168
10000
1615587382.3157017


KeyboardInterrupt: 

In [105]:
text_sep_fin[0]

['The pure spinor formulation of the ten-dimensional superstring leads to manifestly supersymmetric loop amplitudes, expressed as integrals in pure spinor superspace',
 ' This paper explores different methods to evaluate these integrals and then uses them to calculate the kinematic factors of the one-loop and two-loop massless four-point amplitudes involving two and four Ramond states']